In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/task-ocrrrr/ocr document (1).pdf


In [2]:
!pip install PyPDF2 pdf2image pytesseract tabula-py psutil

import psutil
import time
import os
import gc
from pdf2image import convert_from_path
import pytesseract
from PyPDF2 import PdfReader
from tabula import read_pdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 91.2 MB/s eta 0:00:00:00:010:01


In [3]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler118 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler118 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 72 not upgraded.
Need to get 3427 kB of archives.
After this operation, 17.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 poppler-data all 0.4.11-1 [2171 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.5 [1071 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 3427 kB in 1s (3348 kB/s)      
Selecting previously unselected 

In [4]:
!sudo apt-get install -y tesseract-ocr
!sudo apt-get install -y tesseract-ocr-nep
!pip install Pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 72 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-nep
0 upgraded, 1 newly installed, 0 to remove and 72 not upgraded.
Need to get 478 kB of archives.
After this operation, 1018 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-nep all 1:4.00~git30-7274cfa-1.1 [478 kB]
Fetched 478 kB in 0s (2515 kB/s)          
Selecting previously unselected package tesseract-ocr-nep.
(Reading database ... 123567 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-nep_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-nep (1:4.00~git30-7274cfa-1.1) ...
Setting up tesser

In [5]:
!which tesseract

/usr/bin/tesseract


In [6]:
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract" 

In [7]:
!tesseract --list-langs


List of available languages (3):
eng
nep
osd


In [12]:
def pdf_to_image(pdf_path, first_page, last_page):
    pages = convert_from_path(pdf_path, 300, first_page=first_page, last_page=last_page)
    return pages

def ocr_on_page(page, lang='eng+nep'):
    text = pytesseract.image_to_string(page, lang=lang)
    return text

def extract_tables_from_pdf(pdf_path, page_number):
    try:
        tables = read_pdf(pdf_path, pages=page_number, multiple_tables=True, pandas_options={"header": None})
        return tables
    except Exception as e:
        return []

def save_output(output_file, page_number, text, tables):
    with open(output_file, 'a', encoding='utf-8') as file:
        file.write(f"Page {page_number}:\n")
        if text.strip():
            file.write("Extracted Text:\n")
            file.write(text + "\n\n")
        if tables:
            for i, table in enumerate(tables, start=1):
                file.write(f"Table {i}:\n")
                file.write(table.to_string(index=False) + "\n\n")


In [13]:
def process_pdf_in_batches(pdf_path, total_pages, output_file='output.txt', pages_per_batch=50, lang='eng+nep'):
    # Initialize the output file (write mode)
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write("PDF Processing Output\n\n")

    batch_count = 1
    for start_page in range(1, total_pages + 1, pages_per_batch):
        end_page = min(start_page + pages_per_batch - 1, total_pages)
        print(f"Processing batch {batch_count}: Pages {start_page} to {end_page}")

        pages = pdf_to_image(pdf_path, start_page, end_page)

        for page_number, page in enumerate(pages, start=start_page):
            text = ocr_on_page(page, lang)
            print(f"OCR text for page {page_number}: {text[:100]}...")  # Preview text

            tables = extract_tables_from_pdf(pdf_path, page_number)
            print(f"Extracted tables for page {page_number}: {len(tables)} tables")

            # Save the output to the single output file
            save_output(output_file, page_number, text, tables)

        gc.collect()

        print(f"Batch {batch_count} processing completed.\n")
        batch_count += 1

    print(f"PDF processing completed. Output saved to {output_file}")


In [14]:
def track_performance(func, *args, **kwargs):
    start_time = time.time()
    process = psutil.Process(os.getpid())
    memory_before = process.memory_info().rss

    result = func(*args, **kwargs)

    memory_after = process.memory_info().rss
    end_time = time.time()

    elapsed_time = end_time - start_time
    memory_used = memory_after - memory_before

    return result, elapsed_time, memory_used

In [15]:
pdf_path = "/kaggle/input/task-ocrrrr/ocr document (1).pdf"
output_file = "/kaggle/working/output.txt"

with open(pdf_path, "rb") as f:
    pdf_reader = PdfReader(f)
    total_pages = len(pdf_reader.pages)

result, speed, memory = track_performance(
    process_pdf_in_batches, pdf_path, total_pages, output_file=output_file, pages_per_batch=50
)

print(f"Processing Speed: {speed:.2f} seconds")
print(f"Memory Usage: {memory / (1024 * 1024):.2f} MB")

Processing batch 1: Pages 1 to 50
OCR text for page 1:  

 

— कार्यालय
बालुवाटार, काठमाडौं ।

NO फोन नं.: ४४१९८०४%५
Web Site: www.nrb.org.np

पोष्ट बक्स:७...
Extracted tables for page 1: 0 tables
OCR text for page 2: नेपाल राष्ट्र बैंकबाट “क”, wg” र नपा? वर्गका
इजाजतपत्रप्राप्त संस्थाहरुलाई जारी गरिएको

एकीकृत निर्द...
Extracted tables for page 2: 0 tables
OCR text for page 3: नेपाल राष्ट्र बैंकबाट “क”, “ख” र “ग” वर्गका
इजाजतपत्रप्राप्त संस्थाहरुलाई जारी गरिएको

एकीकृत निर्दे...
Extracted tables for page 3: 0 tables
OCR text for page 4: विषय-सूची

क्र.सं. विषय

१०.
११.
१२.
१३.

न्यूनतम पुँजीकोष सम्बन्धी व्यवस्था
कर्जा प्रवाह, वर्गीकरण ...
Extracted tables for page 4: 0 tables
OCR text for page 5: नेपाल राष्ट्र बैंक
केन्द्रीय कार्यालय
बैंक तथा वित्तीय संस्था नियमन विभाग

इ.प्रा.निर्देशन नं. १/०८०...
Extracted tables for page 5: 1 tables
OCR text for page 6: 1८40 मार्गदर्शन सम्बन्धमा

इजाजतपत्रप्राप्त बैंक तथा वित्तीय संस्थाले यस बैंकबाट जारी गरिएको ए॥€1॥ C...
Extracted tables for page 